# CLARA FINE-TUNING - Conservative Settings (v2)

**Fixed issues from v1:**
- Lower learning rate (5e-5 instead of 2e-4)
- Fewer epochs (1 instead of 3)
- Higher max_grad_norm (1.0 instead of 0.3)
- Proper inference mode for testing

**Strategy:**
- **Personality** → **Mistral 7B**
- **Knowledge** → **Phi-3-mini**

## Cell 1: Setup

In [ ]:
!pip install -q transformers datasets accelerate wandb bitsandbytes
!pip install -q peft trl sentencepiece

import wandb
wandb.login()

from google.colab import drive
drive.mount('/content/drive')

import torch
print(f"✓ Setup complete!")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

## Cell 2: Configuration - CHANGE DIMENSION HERE!

In [ ]:
# ============================================================
# WHICH DIMENSION ARE YOU TRAINING?
# ============================================================

DIMENSION = "warmth"  # <-- CHANGE THIS EACH RUN

# ============================================================
# AUTO-CONFIGURATION (don't edit below)
# ============================================================
PERSONALITY_DIMS = ["warmth", "playful", "formal", "encouragement"]
KNOWLEDGE_DIMS = ["medical", "coding", "teaching", "quantum"]

if DIMENSION in PERSONALITY_DIMS:
    BASE_MODEL = "mistralai/Mistral-7B-Instruct-v0.3"
    DATA_PATH = f"/content/drive/MyDrive/Lily/training_data/{DIMENSION}_training.json"
    model_prefix = "mistral"
    TRAINING_TYPE = "Personality"
elif DIMENSION in KNOWLEDGE_DIMS:
    BASE_MODEL = "microsoft/Phi-3-mini-4k-instruct"
    DATA_PATH = f"/content/drive/MyDrive/Lily/training_data/{DIMENSION}_knowledge.json"
    model_prefix = "phi3"
    TRAINING_TYPE = "Knowledge"
else:
    raise ValueError(f"Unknown dimension: {DIMENSION}")

OUTPUT_DIR = f"/content/drive/MyDrive/Lily/models/{model_prefix}_{DIMENSION}"

# ============================================================
# TRAINING CONFIG - CONSERVATIVE SETTINGS
# ============================================================
EPOCHS = 1              # Reduced from 3
BATCH_SIZE = 4
GRADIENT_ACCUM = 4
LEARNING_RATE = 5e-5    # Reduced from 2e-4 (4x lower)
MAX_LENGTH = 512
MAX_GRAD_NORM = 1.0     # Increased from 0.3

print("=" * 60)
print(f"🎯 TRAINING: {DIMENSION.upper()} (Conservative Settings)")
print("=" * 60)
print(f"  Type:          {TRAINING_TYPE}")
print(f"  Model:         {BASE_MODEL}")
print(f"  Learning Rate: {LEARNING_RATE} (conservative)")
print(f"  Epochs:        {EPOCHS}")
print(f"  Output:        {OUTPUT_DIR}")
print("=" * 60)

## Cell 3: Check GPU

In [ ]:
!nvidia-smi

import torch
if torch.cuda.is_available():
    print(f"\n✓ GPU: {torch.cuda.get_device_name(0)}")
    print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    raise RuntimeError("No GPU! Go to Runtime → Change runtime type → A100")

## Cell 4: Load Data

In [ ]:
import json
import os
from datasets import Dataset

# Check file exists
if not os.path.exists(DATA_PATH):
    print(f"❌ File not found: {DATA_PATH}")
    print("\nAvailable files:")
    data_dir = "/content/drive/MyDrive/Lily/training_data"
    if os.path.exists(data_dir):
        for f in sorted(os.listdir(data_dir)):
            print(f"  - {f}")
    raise FileNotFoundError(DATA_PATH)

# Load data
with open(DATA_PATH) as f:
    raw_data = json.load(f)

print(f"✓ Loaded {len(raw_data)} examples")

# Format for training
def format_data(examples, dimension, training_type):
    formatted = []
    for ex in examples:
        if training_type == "Personality":
            if 'neutral' in ex and 'high' in ex:
                formatted.append({
                    "instruction": f"Rewrite this with high {dimension}: {ex['neutral']}",
                    "response": ex['high']
                })
            if 'neutral' in ex and 'low' in ex:
                formatted.append({
                    "instruction": f"Rewrite this with low {dimension}: {ex['neutral']}",
                    "response": ex['low']
                })
        else:
            if 'question' in ex and 'answer' in ex:
                formatted.append({
                    "instruction": ex['question'],
                    "response": ex['answer']
                })
    return formatted

formatted = format_data(raw_data, DIMENSION, TRAINING_TYPE)
print(f"✓ Formatted: {len(formatted)} training examples")

# Create dataset
dataset = Dataset.from_list(formatted)
dataset = dataset.train_test_split(test_size=0.1, seed=42)
print(f"  Train: {len(dataset['train'])}, Val: {len(dataset['test'])}")

## Cell 5: Load Model

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

print(f"Loading {BASE_MODEL}...")

model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    quantization_config=bnb_config,
    device_map="auto",
    trust_remote_code=True,
    attn_implementation="eager"
)

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL, trust_remote_code=True)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

print(f"✓ Model loaded: {model.num_parameters():,} parameters")

## Cell 6: Configure LoRA

In [ ]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.1,  # Increased from 0.05 for regularization
    bias="none",
    task_type="CAUSAL_LM"
)

model = prepare_model_for_kbit_training(model)
model = get_peft_model(model, lora_config)

trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
total = sum(p.numel() for p in model.parameters())
print(f"✓ LoRA: {trainable:,} trainable ({100*trainable/total:.2f}%)")

## Cell 7: Tokenize

In [ ]:
def create_prompt(instruction, response=""):
    if response:
        return f"### Instruction:\n{instruction}\n\n### Response:\n{response}"
    return f"### Instruction:\n{instruction}\n\n### Response:\n"

def tokenize_fn(examples):
    prompts = [create_prompt(i, r) for i, r in zip(examples['instruction'], examples['response'])]
    tokenized = tokenizer(prompts, truncation=True, max_length=MAX_LENGTH, padding="max_length")
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

print("Tokenizing...")
tokenized = dataset.map(tokenize_fn, batched=True, remove_columns=dataset['train'].column_names)
print("✓ Done")

## Cell 8: Training Config (CONSERVATIVE)

In [ ]:
from transformers import TrainingArguments
import os

os.makedirs(OUTPUT_DIR, exist_ok=True)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    
    # Training - CONSERVATIVE
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUM,
    
    # Optimizer - LOWER LR
    learning_rate=LEARNING_RATE,
    lr_scheduler_type="cosine",
    warmup_ratio=0.1,
    optim="paged_adamw_8bit",
    weight_decay=0.01,
    
    # Precision
    bf16=True,
    
    # Logging
    logging_steps=10,
    logging_first_step=True,
    
    # Saving
    save_strategy="steps",
    save_steps=50,
    save_total_limit=2,
    
    # Eval
    eval_strategy="steps",
    eval_steps=25,
    
    # W&B
    report_to="wandb",
    run_name=f"clara-{model_prefix}-{DIMENSION}-v2",
    
    # Stability
    gradient_checkpointing=True,
    max_grad_norm=MAX_GRAD_NORM,
)

print("✓ Training config (CONSERVATIVE)")
print(f"  LR: {LEARNING_RATE} (4x lower than default)")
print(f"  Epochs: {EPOCHS}")
print(f"  Max grad norm: {MAX_GRAD_NORM}")

## Cell 9: Train!

In [ ]:
from transformers import Trainer, DataCollatorForLanguageModeling

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized['train'],
    eval_dataset=tokenized['test'],
    data_collator=DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False),
)

print("=" * 60)
print(f"🚀 TRAINING: {DIMENSION.upper()} (Conservative)")
print("=" * 60)
print(f"  Watch: https://wandb.ai/chris_hartline/clara-deng-research")
print("=" * 60)

trainer.train()

print("\n✓ Training complete!")

## Cell 10: Save Model

In [ ]:
print(f"Saving to: {OUTPUT_DIR}")

model.save_pretrained(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

# Size
total_size = sum(
    os.path.getsize(os.path.join(OUTPUT_DIR, f)) 
    for f in os.listdir(OUTPUT_DIR) 
    if os.path.isfile(os.path.join(OUTPUT_DIR, f))
)
print(f"✓ Saved ({total_size/1e6:.1f} MB)")

## Cell 11: Test Model (Fixed)

In [ ]:
print("\n" + "=" * 60)
print("TESTING TRAINED MODEL")
print("=" * 60)

# IMPORTANT: Disable gradient checkpointing for inference
model.eval()
model.config.use_cache = True
if hasattr(model, 'gradient_checkpointing_disable'):
    model.gradient_checkpointing_disable()

def generate_response(prompt, max_new_tokens=100):
    full_prompt = create_prompt(prompt)
    inputs = tokenizer(full_prompt, return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=0.7,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id,
            repetition_penalty=1.1  # Helps prevent repetition
        )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    if "### Response:" in response:
        response = response.split("### Response:")[-1].strip()
    return response

# Test prompts
if TRAINING_TYPE == "Personality":
    test_prompts = [
        f"Rewrite this with high {DIMENSION}: I can help you with that.",
        f"Rewrite this with high {DIMENSION}: That's correct.",
        f"Rewrite this with low {DIMENSION}: I'm happy to help!",
    ]
else:
    test_prompts = [
        f"Explain a basic concept in {DIMENSION}.",
        f"What should a beginner know about {DIMENSION}?",
    ]

print("\nResults:\n")
all_ok = True
for prompt in test_prompts:
    print(f"Prompt: {prompt}")
    response = generate_response(prompt)
    print(f"Response: {response}\n")
    
    # Check for garbled output
    if "Question:" in response or "}}" in response or "QR" in response:
        print("⚠️  WARNING: Output looks garbled!")
        all_ok = False
    print("-" * 40)

if all_ok:
    print("\n✅ Model looks good!")
else:
    print("\n⚠️  Model may need retraining with even lower LR")

## Cell 12: Next Steps

In [ ]:
wandb.finish()

# Check progress
import os
models_dir = "/content/drive/MyDrive/Lily/models"

print("\n" + "=" * 60)
print("TRAINING PROGRESS")
print("=" * 60)

all_dims = PERSONALITY_DIMS + KNOWLEDGE_DIMS
completed = 0

for dim in all_dims:
    prefix = "mistral" if dim in PERSONALITY_DIMS else "phi3"
    path = f"{models_dir}/{prefix}_{dim}"
    exists = os.path.exists(path)
    icon = "✅" if exists else "⬜"
    print(f"  {icon} {dim} ({prefix})")
    if exists:
        completed += 1

print(f"\nProgress: {completed}/8")

if completed < 8:
    remaining = [d for d in all_dims if not os.path.exists(
        f"{models_dir}/{'mistral' if d in PERSONALITY_DIMS else 'phi3'}_{d}"
    )]
    print(f"\n👉 Next: Change DIMENSION = '{remaining[0]}' and run again")